# Continuous Wave (CW) measurements

This notebook will serve as a simple guide to performing CW measurements to characterize the electron shelving fluxonium qubit. First, we import the dependencies for the notebook, to help with the data analysis.

In [3]:
from plottr.data.datadict_storage import datadict_from_hdf5
from plottr.data.datadict_storage import DataDict, DDH5Writer
from plottr.data.datadict_storage import search_datadict
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import copy
import itertools
# import scqubits as scq
import csv
from sklearn.decomposition import PCA
from scipy.signal import find_peaks
from lmfit import Model
from scipy.optimize import curve_fit

The following function is a search function. Throughout all of our experiments, we will be datataking by saving the data to a logs folder as ddr5 files. The following function helps search and put our data into dictionaries for easy visualization.

In [4]:
basedir = r"D:/Redza/Logs"

#Fido function to obtain monitr files
def Fido(datetime, name):          #Fido gets your raw _data for you!
    foldername, datadict = search_datadict(basedir, datetime, name=name , newest = True, only_complete=False)
    print(foldername)
    print(datadict)
    return foldername, datadict

## Characterizing the qubit spectrum

In a shelving fluxonium qubit, the initial characterization step is to perform a bias sweep, going through different magnetic flux bias current values, while a VNA sweeps the frequency of a probe signal that is reflected off the filter port of the qubit. Skipping over some of the more mathematical details, what is obtained is a colormap which highlights the transition frequencies of the qubit at various bias values. 

After proper setup of the setup_cw.py file, run the cw_shelvingreadout_biasSweep.py file, and view it through plottr-monitr. If the results look good, we can proceed to fitting the spectrum to determine the qubit energies.

From these qubit energies, we then can perform a power sweep around the expected qubit shelving frequency. Run the cw_punchout.py file (it's not really a punchout experiment, but the style is close enough). A saturation peak should be visible in the results. There is a drop in the $S_{11}$ value around the frequency, which get less and less visible as the power of the probe drive is increased.

The rest of the qubit energies are obtained from the fitted spectrum, but we also may test for the actual transitions using a twotone experiment. To do this, we probe the qubit at the shelving frequency, with a suitable qubit power. Simultaneously, we have another local oscillator (LO) driving the qubit continuously through the drive port. The LO drive is swept in both frequency and power. 

Aside from confirming our qubit spectrum, this also serves to identify multi-photon transitions.

## Characterizing the JPA

The JPA is characterized by an initial sweep of the bias into the JPA itself. We probe the qubit (line) with a probe signal from the VNA while sweeping the bias. This changes the resonance frequency of the JPA gradually, which in turn changes the gain profile of the JPA. We determine the JPA bias value which causes a resonance peak at the frequency we wish to amplify.

Next, after determining the optimal frequency of the JPA, we pump the JPA's pump line at double the resonance frequency using another LO. The power of the pump drive is swept, and we look at the gain of frequencies.